In [ ]:
import pandas as pd
import numpy as np
import  math
from statistics import mean
from statistics import stdev
import matplotlib.pyplot as plt
import geopandas as gpd
from scipy.stats import norm
import statistics as stat
from scipy.stats import pearsonr
import random
import contextily as ctx
import seaborn.palettes
import seaborn.utils
import warnings

PROJ: proj_create_from_database: Cannot find proj.db


## Utilities

This document has three utility function

---

### 1. `calculate_pvalues(df)`
- **Purpose:**  
  Calculates p-values for pairwise Pearson correlation coefficients between numeric columns in a DataFrame.  
- **Why Created:**  
  To assess the statistical significance of correlations in numeric datasets, enabling data analysts to identify relationships that are not due to random chance.  
- **Usage:**  
  Useful for exploratory data analysis to identify meaningful associations between variables. Only numeric columns are analyzed, and missing values are handled by row exclusion.

---

### 2. `mean_std(data)`
- **Purpose:**  
  Computes the mean and standard deviation of a flattened 2D data structure.  
- **Why Created:**  
  Provides a quick statistical summary of 2D numerical datasets, which is valuable for understanding the central tendency and variability in data.  
- **Usage:**  
  Applied during data preprocessing or exploratory analysis, particularly for normalized or structured data stored in 2D lists.

---

### 3. `ProcessData(path)`
- **Purpose:**  
  Loads and preprocesses a dataset containing location-specific high and low variable values.  
- **Why Created:**  
  Facilitates the ingestion of spatial and categorical datasets, converting them into a structured pandas DataFrame for further analysis. This is especially valuable for co-location pattern studies.  
- **Usage:**  
  Parses a CSV file containing geospatial data, standardizes column headers, and ensures appropriate data types for latitude, longitude, and variable values. Designed for workflows involving spatial data analysis and co-location mining.

---


In [ ]:
def calculate_pvalues(df):
    """
    Calculate the p-values for pairwise Pearson correlation coefficients between numeric columns in a DataFrame.

    Parameters:
    df (pandas.DataFrame): The input DataFrame containing numeric data.

    Returns:
    pandas.DataFrame: A DataFrame containing the p-values of pairwise Pearson correlations.
                       Each cell [r, c] represents the p-value for the correlation between
                       column r and column c in the input DataFrame.

    Notes:
    - The function drops rows with missing values before computation.
    - Only numeric columns are considered for the analysis.
    """
    # Drop rows with missing values and select only numeric columns
    df = df.dropna()._get_numeric_data()

    # Create an empty DataFrame to store p-values
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')

    # Calculate p-values for each pair of columns
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(pearsonr(df[r], df[c])[1], 4)

    return pvalues


In [ ]:
def mean_std(data):
    """
    Calculate the mean and standard deviation of a flattened 2D data structure.

    Parameters:
    data (list of lists): A 2D list containing numerical values.

    Returns:
    tuple: A tuple containing:
        - datamean (float): The mean of the flattened data.
        - datastd (float): The standard deviation of the flattened data.
    """
    # Flatten the 2D data into a single list
    flat_data = [element for item in data for element in item]

    # Calculate mean and standard deviation
    datastd = stat.stdev(flat_data)
    datamean = stat.mean(flat_data)

    return datamean, datastd


In [1]:
def ProcessData(path):
    """
    Load and preprocess a dataset containing locationwise high and low variable values.

    Parameters:
    path (str): The file path to the CSV file to be processed.

    Returns:
    pandas.DataFrame: A DataFrame containing the processed co-location data with specified column headers and data types.

    Column Details:
    - Zip (str): Identifier for the location (e.g., zip code).
    - lat (float64): Latitude values.
    - lon (float64): Longitude values.
    - high (float64): Values for the high variable.
    - low (float64): Values for the low variable.
    """
    # Define column headers and data types for the dataset
    headers = ['Zip', 'lat', 'lon', 'high', 'low']
    dtypes = {'Zip': 'str', 'lat': 'float64', 'lon': 'float64', 'high': 'float64', 'low': 'float64'}

    # Load the CSV file into a DataFrame with the specified headers and data types
    data = pd.read_csv(path, names=headers, dtype=dtypes)

    return data


## Main Implementation Functions Summary

This summary outlines the implementation of key spatial analysis functions designed for influence density calculation and co-location analysis. These functions provide foundational tools for spatial pattern discovery and analysis.

---

### 1. `influence(data, pattern, xx, yy, bandwidth, n)`
- **Purpose:**  
  Calculates the influence density of a given pattern over a spatial grid using Gaussian Kernel Density Estimation (KDE).
- **Parameters:**  
  - `data`: List of data points with spatial coordinates and pattern values.  
  - `pattern`: Index representing the pattern value in the data points.  
  - `xx`, `yy`: 2D grids of longitudes and latitudes for density evaluation.  
  - `bandwidth`: Bandwidth parameter for smoothing in Gaussian KDE.  
  - `n`: Total number of data points (recalculated internally).  
- **Output:**  
  - `density_2d`: A 2D list of influence densities for each grid point.  
- **Key Note:**  
  Useful for detecting spatial hotspots of a specific pattern using density evaluation.

---

### 2. `influenceO(data, pattern, xx, yy, bandwidth, n)` (Hypothetical Comparison)
- **Key Difference from `influence`:**  
  - While `influence` uses Gaussian KDE, `influenceO` may employ an alternative way of calculation.

---

### 3. `equationC3NoLog(data1, data2, data3, xdimention, ydimention, xx, yy)`
- **Purpose:**  
  Calculates co-location values across a grid using normalized Z-scores for three spatial density datasets.
- **Parameters:**  
  - `data1`, `data2`, `data3`: 2D density data for three spatial variables.  
  - `xdimention`, `ydimention`: Dimensions of the spatial grid.  
  - `xx`, `yy`: 2D grids of longitudes and latitudes for analysis.  
- **Output:**  
  - Co-location values represented as a 2D grid, highlighting areas where the three variables exhibit significant interaction or association.  
- **Usage:**  
  Useful for multivariate spatial analysis to identify regions with complex co-location patterns.

---

### 4. `equationC2NoLog(data1, data2, xdimention, ydimention, xx, yy)`
- **Purpose:**  
  Calculates co-location values for two spatial density datasets using normalized Z-scores.
- **Parameters:**  
  - `data1`, `data2`: 2D density data for two spatial variables.  
  - `xdimention`, `ydimention`: Dimensions of the spatial grid.  
  - `xx`, `yy`: 2D grids of longitudes and latitudes for analysis.  
- **Output:**  
  - Co-location values as a 2D grid, illustrating pairwise spatial interaction between two variables.  
- **Usage:**  
  Focuses on pairwise spatial associations, offering simpler but more focused spatial analysis compared to `equationC3NoLog`.

---
### 5. `generate_sample_points(data)`
- **Purpose:**  
  Generates a grid of sample points within the bounding box defined by the dataset's minimum and maximum longitude and latitude values.
- **Parameters:**  
  - `data`: A dictionary containing keys `'lon'` and `'lat'`, each associated with a list of longitude and latitude values, respectively.  
- **Output:**  
  - `xx`: 2D array of longitude values representing the grid.  
  - `yy`: 2D array of latitude values representing the grid.  
  - `grid`: A list of `[longitude, latitude]` pairs for each grid point.  
- **Key Note:**  
  Forms the foundational grid for spatial density and co-location analysis.
  ---
### Key Differences
- **`influence` vs `influenceO`:**  
  The main difference lies in the kernel estimation methods or weighting mechanisms. `influence` uses Gaussian KDE, while `influenceO` uses an alternative way of calculation.
- **`equationC3NoLog` vs `equationC2NoLog`:**  
  `equationC3NoLog` evaluates co-locations for three variables simultaneously, highlighting complex spatial interactions. `equationC2NoLog` focuses on simpler pairwise co-locations.



In [ ]:
def influence(data, pattern, xx, yy, bandwidth, n):
    """
    Calculate the influence density of a given pattern across a spatial grid using Gaussian kernel density estimation.

    Parameters:
    data (list): A list of data points, where each data point contains spatial coordinates and pattern values.
    pattern (int): Index of the pattern value in the data points.
    xx (list of lists): 2D grid of longitudes for density evaluation.
    yy (list of lists): 2D grid of latitudes for density evaluation.
    bandwidth (float): Bandwidth parameter for Gaussian kernel smoothing.
    n (int): Total number of data points (redundant as it is recalculated internally).

    Returns:
    density_2d (list of lists): A 2D list containing influence densities at each grid point.

    """
    # Recalculate the number of data points (n) from the input dataset
    n = len(data)

    # Initialize variables for density calculation
    xcount = 0
    density_2d = []  # 2D list to store density values
    x = []  # List to store longitude values
    y = []  # List to store latitude values
    d = []  # List to store calculated density values

    # Iterate through the longitude grid
    for longitude in xx:
        temp = []  # Temporary list to store density values for the current longitude row
        ycount = 0  # Latitude index

        # Iterate through the latitude grid
        for latitude in yy:
            sum_probability = 0  # Sum of weighted probabilities for the current grid point

            # Store grid coordinates
            x.append(xx[xcount][xcount])
            y.append(yy[ycount][ycount])

            # Calculate the influence density for the current grid point
            for item in data:
                # Compute squared distances in longitude and latitude
                xdistance = (item[1] - longitude[xcount]) ** 2
                ydistance = (item[0] - latitude[ycount]) ** 2

                # Compute combined distance and apply Gaussian kernel formula
                combined_distance = xdistance + ydistance
                probability = math.exp(-1 * combined_distance / (2 * bandwidth ** 2))

                # Weight the probability by the pattern value
                weighted_probability = item[pattern] * probability
                sum_probability += weighted_probability

            # Normalize the probability by the kernel area and total number of points
            kernel_area = 2 * math.pi * bandwidth ** 2
            average_probability = sum_probability / (n * kernel_area)

            # Append the density value
            d.append(average_probability)
            temp.append(average_probability)
            ycount += 1

        # Append the row of density values to the 2D density list
        density_2d.append(temp)
        xcount += 1

    # Create a dictionary to store the results in tabular format
    density_dict = {'X': x, 'Y': y, 'Colocation': d}

    # Convert the dictionary to a Pandas DataFrame and save it to a CSV file
    #df = pd.DataFrame(data=density_dict)
    #df.to_csv('UnemploymentLow.csv', index=False)

    # Return the 2D density grid
    return density_2d


In [ ]:
def influenceO(data, pattern, xx, yy, bandwidth):
    """
    Calculate the influence density of a given pattern across a spatial grid using Gaussian kernel density estimation.

    Parameters:
    data (list): A list of data points, where each data point contains spatial coordinates and pattern values.
    pattern (int): Index of the pattern value in the data points.
    xx (list of lists): 2D grid of longitudes for density evaluation.
    yy (list of lists): 2D grid of latitudes for density evaluation.
    bandwidth (float): Bandwidth parameter for Gaussian kernel smoothing.

    Returns:
    density_2d (list of lists): A 2D list containing influence densities at each grid point.
    """
    # Recalculate the number of data points (n) from the input dataset
    n = len(data)

    # Precompute constants for efficiency
    kernel_area = 2 * math.pi * bandwidth ** 2

    # Initialize variables for density calculation
    density_2d = []  # 2D list to store density values

    # Iterate through the longitude grid
    for longitude_row in xx:
        temp = []  # Temporary list to store density values for the current longitude row

        # Iterate through the latitude grid
        for latitude_row in yy:
            sum_probability = 0  # Sum of weighted probabilities for the current grid point

            # Calculate the influence density for the current grid point
            for item in data:
                # Compute squared distances in longitude and latitude
                xdistance = (item[1] - longitude_row) ** 2
                ydistance = (item[0] - latitude_row) ** 2

                # Compute combined distance and apply Gaussian kernel formula
                combined_distance = xdistance + ydistance
                probability = math.exp(-1 * combined_distance / (2 * bandwidth ** 2))

                # Weight the probability by the pattern value
                weighted_probability = item[pattern] * probability
                sum_probability += weighted_probability

            # Normalize the probability by the kernel area and total number of points
            average_probability = sum_probability / (n * kernel_area)

            # Append the density value
            temp.append(average_probability)

        # Append the row of density values to the 2D density list
        density_2d.append(temp)

    # Return the 2D density grid
    return density_2d


In [ ]:
def equationC3NoLog(data1, data2, data3, xdimention, ydimention, xx, yy):
    """
    Calculate the co-location value for three densities using the normalized Z-scores of three datasets.

    Parameters:
    data1 (list of lists): Density data for the first variable.
    data2 (list of lists): Density data for the second variable.
    data3 (list of lists): Density data for the third variable.
    xdimention (int): Number of grid points along the x-axis.
    ydimention (int): Number of grid points along the y-axis.
    xx (list of lists): 2D grid of longitudes.
    yy (list of lists): 2D grid of latitudes.

    Returns:
    Co-location on a grid (list of lists): A 2D list containing the calculated co-location values at each grid point.
    """
    # Flatten data1 to calculate mean and standard deviation
    flat_data1_density = [element for item in data1 for element in item]
    mean_data1 = stat.mean(flat_data1_density)
    std_data1 = stat.stdev(flat_data1_density)

    # Flatten data2 to calculate mean and standard deviation
    flat_data2_density = [element for item in data2 for element in item]
    mean_data2 = stat.mean(flat_data2_density)
    std_data2 = stat.stdev(flat_data2_density)

    # Flatten data3 to calculate mean and standard deviation
    flat_data3_density = [element for item in data3 for element in item]
    mean_data3 = stat.mean(flat_data3_density)
    std_data3 = stat.stdev(flat_data3_density)

    # Initialize variables
    count_x = 0
    C3 = []  # 2D list to store C3 values
    x = []   # List to store longitude values
    y = []   # List to store latitude values
    d = []   # List to store C3 values

    # Calculate C3 values for each grid point
    while count_x < xdimention:
        count_y = 0
        temp_C3 = []  # Temporary list for the current row of C3 values

        while count_y < ydimention:
            # Calculate Z-scores for the three datasets
            Z_A = (data1[count_x][count_y] - mean_data1) / std_data1
            Z_B = (data2[count_x][count_y] - mean_data2) / std_data2
            Z_C = (data3[count_x][count_y] - mean_data3) / std_data3

            # Store coordinates
            x.append(xx[count_x][count_x])
            y.append(yy[count_y][count_y])

            # Calculate C3 value only if all Z-scores are positive
            data = 0
            if Z_A > 0 and Z_B > 0 and Z_C > 0:
                data = (Z_A * Z_B * Z_C) ** (1/3)

            temp_C3.append(data)
            d.append(data)
            count_y += 1

        C3.append(temp_C3)
        count_x += 1

    return C3


In [2]:
def equationC2NoLog(data1, data2, xdimention, ydimention, xx, yy):
    """
    Calculate the co-location value for two densities using the normalized Z-scores of two datasets.

    Parameters:
    data1 (list of lists): Density data for the first variable.
    data2 (list of lists): Density data for the second variable.
    xdimention (int): Number of grid points along the x-axis.
    ydimention (int): Number of grid points along the y-axis.
    xx (list of lists): 2D grid of longitudes.
    yy (list of lists): 2D grid of latitudes.

    Returns:
    Co-location on a grid (list of lists): A 2D list containing the calculated co-location values at each grid point.
    """
    # Flatten data1 to calculate mean and standard deviation
    flat_data1_density = [element for item in data1 for element in item]
    mean_data1 = stat.mean(flat_data1_density)
    std_data1 = stat.stdev(flat_data1_density)

    # Flatten data2 to calculate mean and standard deviation
    flat_data2_density = [element for item in data2 for element in item]
    mean_data2 = stat.mean(flat_data2_density)
    std_data2 = stat.stdev(flat_data2_density)

    # Initialize variables
    count_x = 0
    C2 = []  # 2D list to store C2 values
    x = []   # List to store longitude values
    y = []   # List to store latitude values
    d = []   # List to store C2 values

    # Calculate C2 values for each grid point
    while count_x < xdimention:
        count_y = 0
        temp_C2 = []  # Temporary list for the current row of C2 values

        while count_y < ydimention:
            # Calculate Z-scores for the two datasets
            Z_A = (data1[count_x][count_y] - mean_data1) / std_data1
            Z_B = (data2[count_x][count_y] - mean_data2) / std_data2

            # Store coordinates
            x.append(xx[count_x][count_x])
            y.append(yy[count_y][count_y])

            # Calculate C2 value only if both Z-scores are positive
            data = 0
            if Z_A > 0 and Z_B > 0:
                data = math.sqrt(Z_A * Z_B)

            temp_C2.append(data)
            d.append(data)
            count_y += 1

        C2.append(temp_C2)
        count_x += 1

    # Save the results to a CSV file
    d2 = {'X': x, 'Y': y, 'Colocation': d}
    df2 = pd.DataFrame(data=d2)
    df2.to_csv('Anukriti/Covid_High_Unemployment_High.csv', index=False)

    return C2


In [ ]:
def generate_sample_points(data):
    """
    Generate a grid of sample points within the bounding box defined by the minimum and maximum
    longitude and latitude values from the dataset.

    Parameters:
    data (dict): A dictionary containing 'lon' and 'lat' keys, each associated with a list of longitude
                 and latitude values respectively.

    Returns:
    xx (numpy.ndarray): 2D array of longitude values representing the grid.
    yy (numpy.ndarray): 2D array of latitude values representing the grid.
    grid (list of lists): A list containing [longitude, latitude] pairs for each grid point.
    """
    # Extract the bounding box for the grid
    xmin = min(data['lon'])
    xmax = max(data['lon'])
    ymin = min(data['lat'])
    ymax = max(data['lat'])

    # Generate a meshgrid with 50 equally spaced points between the min and max coordinates
    xx, yy = np.mgrid[xmin:xmax:50j, ymin:ymax:50j]

    # Extract unique x-coordinates from the meshgrid for easier indexing
    X = []
    count = 0
    for item in xx:
        X.append(item[count])
        count += 1

    # Flatten the meshgrid into a list of [longitude, latitude] pairs
    grid = []
    xcount = 0
    for longitude in xx:
        ycount = 0
        for latitude in yy:
            grid.append([longitude[xcount], latitude[ycount]])
            ycount += 1
        xcount += 1

    return xx, yy, grid


## Analysis Guideline

This guideline outlines the key steps for conducting spatial data analysis, focusing on influence density calculation, co-location analysis, and statistical testing.

---

### 1. **Read the Data Files**
   - Load the required spatial datasets containing coordinates, variables, and patterns.
   - Ensure data quality by handling missing values and standardizing formats (e.g., CSV files with proper headers).

---

### 2. **Generate Sample Points**
   - Use the `generate_sample_points` function to create a grid of sample points:
     - Input spatial data with longitude (`lon`) and latitude (`lat`) values.
     - Generate a bounding box based on the dataset's minimum and maximum longitude/latitude values.
     - Output includes:
       - `xx`: 2D array of longitude values for the grid.
       - `yy`: 2D array of latitude values for the grid.
       - `grid`: List of `[longitude, latitude]` pairs for each grid point.
   - This step establishes the foundation for spatial density and co-location analysis by defining the spatial grid.

---

### 3. **Calculate Influence Function**
   - Use the `influence` or `influenceO` function to compute spatial influence densities for each pattern:
     - Input spatial coordinates, pattern indices, grid dimensions (`xx`, `yy`), and kernel bandwidth.
     - Generate a 2D density map representing the influence of each pattern across the grid.
   - Consider the choice between `influence` and `influenceO` based on the specific kernel or context of the analysis.

---

### 4. **Calculate Binary or Ternary Co-locations**
   - For **binary co-locations**:
     - Use `equationC2NoLog` to compute co-location values for two variables.
   - For **ternary co-locations**:
     - Use `equationC3NoLog` to compute co-location values involving three variables.
   - Input parameters include density data, grid dimensions, and the spatial grid (`xx`, `yy`).

---

### 5. **Visualize the Outputs**
   - Generate clear and interpretable visualizations:
     - Plot influence densities as heatmaps or contour plots.
     - Visualize co-location results to highlight spatial patterns of variable interaction.
   - Use consistent color schemes and annotations for easy interpretation.

---

### 6. **Save the Co-locations**
   - Export computed co-location values to files (e.g., CSV or JSON) for documentation and further analysis.
   - Include metadata such as variable names, grid dimensions, and calculation parameters.

---

### 7. **Perform Permutation Test**
   - Conduct permutation tests to validate the statistical significance of co-location patterns:
     - Randomly shuffle spatial patterns and recompute co-locations.
     - Compare observed co-location values with randomized distributions to compute p-values.
   - Save permutation test results for reference and reporting.

---



### Read Data

In [4]:
"""
This code processes multiple datasets related to high and low values of various variables,
such as COVID cases, population, income, and unemployment rates. The processed datasets
are converted into NumPy arrays for further analysis and stored in a list for structured handling.

Purpose:
- Load and process CSV files for each variable, dropping irrelevant columns (e.g., 'Zip').
- Convert processed data into NumPy arrays for numerical computation and analysis.
- Store the processed datasets in a list for ease of access.

Key Steps:
1. Use the `ProcessData` function to read and clean each dataset.
2. Drop the 'Zip' column as it's irrelevant for numerical operations.
3. Append the processed DataFrame to the `dat` list for centralized storage.
4. Convert the processed DataFrame to a NumPy array for numerical computations.
"""

dat = []  # List to store processed datasets

# Process COVID data
data = ProcessData("Datasets/covid_high_low.csv")
dat.append(data)
covid = np.array(data.drop(['Zip'], 1))  # Convert to NumPy array

# Process Population data
data = ProcessData("Datasets/population_high_low.csv")
dat.append(data)
population = np.array(data.drop(['Zip'], 1))  # Convert to NumPy array
print(data.head)  # Print the first few rows for verification

# Process Bachelor's Degree data
data = ProcessData("Datasets/bachelorPercentage_high_low.csv")
dat.append(data)
bachelor = np.array(data.drop(['Zip'], 1))  # Convert to NumPy array

# Process Median Income data
data = ProcessData("Datasets/median_income_high_low.csv")
dat.append(data)
median_income = np.array(data.drop(['Zip'], 1))  # Convert to NumPy array

# Process Poverty data
data = ProcessData("Datasets/below_poverty_percent_high_low.csv")
dat.append(data)
poverty = np.array(data.drop(['Zip'], 1))  # Convert to NumPy array

# Process Four Plus Family data
data = ProcessData("Datasets/four_plus_family_percent_high_low.csv")
dat.append(data)
family = np.array(data.drop(['Zip'], 1))  # Convert to NumPy array

# Process No Car data
data = ProcessData("Datasets/no_car_percent_high_low.csv")
dat.append(data)
car = np.array(data.drop(['Zip'], 1))  # Convert to NumPy array

# Process Unemployment data
data = ProcessData("Datasets/unemployment_percent_high_low.csv")
dat.append(data)
unemployment = np.array(data.drop(['Zip'], 1))  # Convert to NumPy array


NameError: name 'pd' is not defined

### Calculate Influence Functions

In [ ]:
"""
This code initializes influence values for various attributes (e.g., COVID rates, population, income) at high and low levels.

Purpose:
- To compute and store influence densities for a range of datasets using a Gaussian kernel approach.
- Organize the results into a structured format for further analysis.

Key Steps:
1. Generate sample points (grid) using the `generate_sample_points` function.
2. Initialize arrays to store influence results (`da`) and their corresponding labels (`str`).
3. Compute influence densities for each attribute and store the results in `da`.
4. Assign descriptive labels for each attribute's high and low values into `str`.
"""

# Set the kernel bandwidth for influence calculation
bandwidth = 1

# Generate sample points from the COVID dataset
xx, yy, grid = generate_sample_points(covid)

# Initialize arrays to store influence results and their labels
da = [0] * 16
str = [""] * 16

# Compute influence densities and assign labels

da[0] = influence(covid, 2, xx, yy, bandwidth)
str[0] = "covid_high"

da[1] = influence(covid, 3, xx, yy, bandwidth)
str[1] = "covid_low"

da[2] = influence(population, 2, xx, yy, bandwidth)
str[2] = "population_high"

da[3] = influence(population, 3, xx, yy, bandwidth)
str[3] = "population_low"

da[4] = influence(bachelor, 2, xx, yy, bandwidth)
str[4] = "bachelor_high"

da[5] = influence(bachelor, 3, xx, yy, bandwidth)
str[5] = "bachelor_low"

da[6] = influence(median_income, 2, xx, yy, bandwidth)
str[6] = "income_high"

da[7] = influence(median_income, 3, xx, yy, bandwidth)
str[7] = "income_low"

da[8] = influence(poverty, 2, xx, yy, bandwidth)
str[8] = "poverty_high"

da[9] = influence(poverty, 3, xx, yy, bandwidth)
str[9] = "poverty_low"

da[10] = influence(family, 2, xx, yy, bandwidth)
str[10] = "family_high"

da[11] = influence(family, 3, xx, yy, bandwidth)
str[11] = "family_low"

da[12] = influence(car, 2, xx, yy, bandwidth)
str[12] = "car_high"

da[13] = influence(car, 3, xx, yy, bandwidth)
str[13] = "car_low"

da[14] = influence(unemployment, 2, xx, yy, bandwidth)
str[14] = "unemployment_high"

da[15] = influence(unemployment, 3, xx, yy, bandwidth)
str[15] = "unemployment_low"


TypeError: influence() missing 1 required positional argument: 'n'

### Visualize the Influence Function

In [ ]:
"""
This code generates a geographical heatmap to visualize the spatial distribution of bachelor's degree holders' weighted density.

Purpose:
- To compute the influence density of bachelor's degree holders using a Gaussian kernel.
- Overlay the density on a geographical map of Virginia zip codes.
- Display the heatmap with customized contour levels and a colorbar for better visualization.

Key Steps:
1. Compute the influence density using the `influence` function.
2. Load geographical data from a GeoJSON file.
3. Create a base map of zip code boundaries.
4. Overlay a filled contour plot of density values.
5. Add a resized and labeled colorbar for interpreting density values.
6. Annotate the plot with a title and axis labels.
"""


# Assuming da[4] is valid
bandwidth = 0.25

# Compute influence density
# The influence function calculates weighted density based on spatial data.
data = influence(bachelor, 2, xx, yy, bandwidth)

# Create a plot
fig, ax = plt.subplots(figsize=(10, 8))

# File path to the GeoJSON data containing zip code geometries
data_path = 'C:/Users/mdmah/PycharmProjects/ProfessorEick/ProfessorEick/ACMSIGSPATIAL2020/VA_Zip_Codes_-2354097028237156067.geojson'

# Load the GeoJSON data into a GeoPandas DataFrame
df_places = gpd.read_file(data_path)

# Plot the base map showing zip code boundaries
df_places.plot(ax=ax, linewidth=0.8, edgecolor='black', color='white')

# Define contour levels for the density plot
levels = np.linspace(0, 0.1, 11)

# Add a filled contour plot to visualize the density values
cfset = ax.contourf(xx, yy, data, levels=levels, cmap='coolwarm', alpha=0.7)

# Resize and reposition the colorbar
cax = fig.add_axes([0.91, 0.3, 0.02, 0.4])  # [left, bottom, width, height]
cbar = fig.colorbar(cfset, cax=cax, orientation='vertical')

# Customize the colorbar
cbar.set_label("Weighted Density", fontsize=14)  # Add a label to the colorbar
cbar.ax.tick_params(labelsize=10)  # Adjust the size of the colorbar ticks

# Add title and axis labels to the plot
ax.set_title(r'$\varphi_{\mathrm{Bachelor\ Degree} \uparrow}$', fontsize=14)
ax.set_xlabel("Longitude", fontsize=14)
ax.set_ylabel("Latitude", fontsize=14)

# Display the plot
plt.show()


### Ternary Co-location and Stastistics Calculation

In [ ]:
"""
This code computes the mean and standard deviation of density values generated for unique patterns of three datasets.

Purpose:
- To iterate through combinations of three datasets from a collection of 16 datasets.
- Compute a density matrix for each combination using the `equationC3NoLog` function.
- Calculate and store the mean and standard deviation for the density values of each combination.
- Record the pattern of dataset indices used for each calculation.

Key Steps:
1. Iterate through all unique combinations of three datasets from 16 datasets.
2. For each combination:
   - Compute the density matrix using `equationC3NoLog`.
   - Calculate the mean and standard deviation using the `mean_std` function.
   - Store the results and the corresponding pattern.
3. Store the aggregated results in lists for further use.
"""

# Initialize variables to store results
i = 0
mean = []
std = []
patterns = []

# Iterate through unique combinations of three datasets
while i <= 15:
    j = i + 1
    while j <= 15:
        k = j + 1
        while k < 15:
            # Temporary lists to store mean and standard deviation for the current combination
            mi = []
            st = []

            # Compute the density matrix for the current combination
            data = equationC3NoLog(da[i], da[j], da[k], 50, 50, xx, yy)

            # Calculate the mean and standard deviation for the density values
            m1, s1 = mean_std(data)
            mi.append(m1)
            st.append(s1)

            # Store the rounded mean and standard deviation
            mean.append(round(stat.mean(mi), 3))
            std.append(round(stat.mean(st), 3))

            # Record the pattern of dataset indices
            patterns.append(str[i] + "+" + str[j] + "+" + str[k])

            k += 1
        j += 1
    i += 1


NameError: name 'equationC3NoLog' is not defined

### Visualize Ternary Co-location

In [ ]:
"""
This code visualizes co-location patterns between two variables (population and median income) using a filled contour plot over a geographical map.

Purpose:
- To compute the co-location values for population and median income using influence and density calculation functions.
- Normalize and visualize the co-location data on a geographical map of Virginia zip codes.
- Provide insights into spatial patterns of the relationship between the two variables.

Key Steps:
1. Compute influence densities for population and median income using a specified bandwidth.
2. Calculate co-location values using the `equationC2NoLog` function.
3. Normalize the co-location values.
4. Overlay the co-location values on a geographical map using a filled contour plot.
5. Customize the visualization with labels, a colorbar, and title.
"""

import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

# Set the kernel bandwidth for influence calculation
bandwidth = 0.01

# Compute influence densities for population and median income
da1 = influence(population, 3, xx, yy, bandwidth)
da2 = influence(median_income, 3, xx, yy, bandwidth)

# Calculate co-location values between population and median income
data = equationC2NoLog(da1, da2, 50, 50, xx, yy)

# Normalize the co-location values
data = [[value / 3 for value in row] for row in data]

# Create a plot
fig, ax = plt.subplots(figsize=(10, 8))

# File path to the GeoJSON data containing zip code geometries
data_path = 'C:/Users/mdmah/PycharmProjects/ProfessorEick/ProfessorEick/ACMSIGSPATIAL2020/VA_Zip_Codes_-2354097028237156067.geojson'

# Load the GeoJSON data into a GeoPandas DataFrame
df_places = gpd.read_file(data_path)

# Plot the base map showing zip code boundaries
df_places.plot(ax=ax, linewidth=0.8, edgecolor='black', color='white')

# Define contour levels for the co-location values
levels = np.linspace(0, 5, 11)

# Add a filled contour plot to visualize the co-location values
cfset = ax.contourf(xx, yy, data, levels=levels, cmap='coolwarm', alpha=0.7)

# Resize and reposition the colorbar
cax = fig.add_axes([0.91, 0.3, 0.02, 0.4])  # [left, bottom, width, height]
cbar = fig.colorbar(cfset, cax=cax, orientation='vertical')

# Customize the colorbar
cbar.set_label("Co-location Value", fontsize=14)  # Add a label to the colorbar
cbar.ax.tick_params(labelsize=10)  # Adjust the size of the colorbar ticks

# Add title and axis labels to the plot
ax.set_title(r'$C_{(\{Population \downarrow,Income \uparrow\}) }$ for Bandwidth 0.01', fontsize=14)
ax.set_xlabel("Longitude", fontsize=14)
ax.set_ylabel("Latitude", fontsize=14)

# Display the plot
plt.show()


### Save Ternary Co-location

In [ ]:
"""
This code saves the calculated co-location results (patterns, mean, and standard deviation) into a CSV file.

Purpose:
- To organize the co-location results into a structured format.
- Save the results as a CSV file for further analysis or reporting.

Key Steps:
1. Create a dictionary with keys:
   - 'pattern': List of dataset index combinations used for co-location calculations.
   - 'mean': List of mean values for the density values of each pattern.
   - 'std': List of standard deviation values for the density values of each pattern.
2. Convert the dictionary into a Pandas DataFrame.
3. Save the DataFrame to a CSV file at the specified path.
"""


# Create a dictionary to store co-location results
d = { 'pattern': patterns, 'mean': mean, 'std': std}

# Convert the dictionary into a DataFrame
df = pd.DataFrame(data=d)

# Save the DataFrame to a CSV file
df.to_csv('Datasets/tarnery_colocations_0_05.csv', index=False)


### Binary Co-location and Stastistics Calculation

In [ ]:
"""
This code computes the mean and standard deviation of density values generated for unique pairwise combinations of datasets.

Purpose:
- To iterate through combinations of two datasets from a collection of 16 datasets.
- Compute a density matrix for each pairwise combination using the `equationC2NoLog` function.
- Calculate and store the mean and standard deviation for the density values of each combination.
- Record the pattern of dataset indices used for each calculation.

Key Steps:
1. Iterate through all unique combinations of two datasets from 16 datasets.
2. For each combination:
   - Compute the density matrix using `equationC2NoLog`.
   - Calculate the mean and standard deviation using the `mean_std` function.
   - Store the results and the corresponding pattern.
3. Store the aggregated results in lists for further use.
"""

# Initialize variables to store results
i = 0
means = []
std = []
patterns = []

# Iterate through unique combinations of two datasets
while i <= 15:
    j = i + 1
    while j <= 15:

        # Temporary lists to store mean and standard deviation for the current combination
        mi = []
        st = []

        # Compute the density matrix for the current combination
        data = equationC2NoLog(da[i], da[j], 50, 50, xx, yy)

        # Calculate the mean and standard deviation for the density values
        m1, s1 = mean_std(data)
        mi.append(m1)
        st.append(s1)

        # Store the rounded mean and standard deviation
        means.append(round(stat.mean(mi), 3))
        std.append(round(stat.mean(st), 3))

        # Record the pattern of dataset indices
        patterns.append(str[i] + "+" + str[j])

        j += 1
    i += 1


### Permutation Test for Binary Co-location

In [ ]:
"""
This code performs a permutation test to calculate p-values for co-location patterns between pairs of datasets.

Purpose:
- To test the significance of co-location patterns by shuffling the values in datasets and recalculating densities.
- Compute p-values based on the distribution of permuted results compared to the original density values.
- Store the p-values, means, and standard deviations of permuted results for further analysis.

Key Steps:
1. Iterate through all unique pairwise combinations of datasets (16 datasets in total).
2. For each combination:
   - Shuffle values in the relevant dataset columns.
   - Recompute influence densities using the `influence` function.
   - Calculate densities for the combination using `equationC2NoLog`.
   - Compute mean and standard deviation of the permuted densities.
   - Append results to lists of p-values, means, and standard deviations.
3. Calculate p-values for higher, lower, and equal distributions of permuted means relative to the original.
"""

import numpy as np
import warnings
import statistics as stat

# Suppress warnings
warnings.filterwarnings('ignore')

# Initialize variables
meanr = []
stdr = []
p_valuesH = []  # p-values for higher distribution
p_valuesL = []  # p-values for lower distribution
p_valuesE = []  # p-values for equal distribution
meansR = []  # Mean of permuted values
stdR = []  # Standard deviation of permuted values

# Iterate through unique combinations of datasets
i = 0
k = 0
while i <= 15:
    j = i + 1
    while j <= 15:
        num_permutations = 100  # Number of permutations
        permuted_vals = []  # List to store results from permutations

        for _ in range(num_permutations):
            # Initialize an empty array for shuffling datasets
            dar = [0] * 16

            # Shuffle and calculate influence for selected datasets
            if i == 0 or j == 0:
                np.random.shuffle(dat[0]['high'])
                d = np.array(dat[0].drop(['Zip'], 1))
                dar[i] = influence(d, 2, xx, yy, bandwidth)
            if i == 1 or j == 1:
                np.random.shuffle(dat[0]['low'])
                d = np.array(dat[0].drop(['Zip'], 1))
                dar[i] = influence(d, 3, xx, yy, bandwidth)
            # Repeat similar blocks for indices 2 through 15

            # Compute the density matrix for the current combination
            data = equationC2NoLog(da[i], da[j], 50, 50, xx, yy)
            m1, s1 = mean_std(data)
            permuted_vals.append(m1)

        # Calculate p-values based on the distribution of permuted values
        p_valueh = np.mean([val > means[k] for val in permuted_vals])
        p_valuesH.append(p_valueh)

        p_valuel = np.mean([val < means[k] for val in permuted_vals])
        p_valuesL.append(p_valuel)

        p_valuee = np.mean([val == means[k] for val in permuted_vals])
        p_valuesE.append(p_valuee)

        # Compute and store statistics of permuted values
        meansR.append(round(stat.mean(permuted_vals), 3))
        stdR.append(stat.stdev(permuted_vals))

        # Print results for the current pattern
        print(patterns[k], p_valueh, p_valuel, p_valuee, round(stat.mean(permuted_vals), 3), stat.stdev(permuted_vals))

        k += 1
        j += 1
    i += 1


covid_high+covid_low 1.0 0.0 0.0 0.028 0.0
covid_high+population_high 1.0 0.0 0.0 0.035 0.0
covid_high+population_low 0.0 1.0 0.0 0.068 0.0
covid_high+bachelor_high 1.0 0.0 0.0 0.055 0.0
covid_high+bachelor_low 0.0 1.0 0.0 0.076 0.0
covid_high+income_high 1.0 0.0 0.0 0.072 0.0
covid_high+income_low 0.0 1.0 0.0 0.069 0.0
covid_high+poverty_high 0.0 1.0 0.0 0.059 0.0
covid_high+poverty_low 0.0 1.0 0.0 0.054 0.0
covid_high+family_high 0.0 1.0 0.0 0.061 0.0
covid_high+family_low 1.0 0.0 0.0 0.065 0.0
covid_high+car_high 1.0 0.0 0.0 0.072 0.0
covid_high+car_low 0.0 1.0 0.0 0.071 0.0
covid_high+unemployment_high 1.0 0.0 0.0 0.049 0.0
covid_high+unemployment_low 0.0 1.0 0.0 0.062 0.0
covid_low+population_high 1.0 0.0 0.0 0.097 0.0
covid_low+population_low 1.0 0.0 0.0 0.161 0.0
covid_low+bachelor_high 0.0 1.0 0.0 0.123 0.0
covid_low+bachelor_low 0.0 1.0 0.0 0.193 0.0
covid_low+income_high 0.0 1.0 0.0 0.157 0.0
covid_low+income_low 1.0 0.0 0.0 0.186 0.0
covid_low+poverty_high 1.0 0.0 0.0 0.109 

### Save the Permutation Test Results

In [ ]:
"""
This code saves the calculated permuted co-location results (patterns, mean, and standard deviation) into a CSV file.

Purpose:
- To organize the co-location results into a structured format.
- Save the results as a CSV file for further analysis or reporting.
- To compare with real results

Key Steps:
1. Create a dictionary with keys:
   - 'pattern': List of dataset index combinations used for co-location calculations.
   - 'mean': List of mean values for the density values of each pattern.
   - 'std': List of standard deviation values for the density values of each pattern.
2. Convert the dictionary into a Pandas DataFrame.
3. Save the DataFrame to a CSV file at the specified path.
"""
# Create a dictionary to store co-location results
d = { 'pattern': patterns, 'mean': mean, 'std':std}

# Convert the dictionary into a DataFrame
df = pd.DataFrame(data=d)

# Save the DataFrame to a CSV file
df.to_csv('Datasets/binary_colocations_random_0_05.csv', index=False)
